In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import ticker
import awkward as ak
import uproot 
import mplhep as hep
import hist
import os
import time
from numba import njit
from matplotlib.colors import LogNorm
import tqdm
import pickle 
from numba import jit



from UtilityAndConst import *
from Functions import *

dir = "/Users/francescolarovere/Documents/Tesi-Triennale/Dati"
os.chdir(dir)

hep.style.use("CMS")
np.set_printoptions(threshold=np.inf)

In [2]:
files = [
    "./ntupleStubsBmtfGmt_LS133.root",
    "./ntupleStubsBmtfGmt_LS263.root",
    "./ntupleStubsBmtfGmt_LS264.root",
    "./ntupleStubsBmtfGmt_LS306.root",
]

stub_bmtf_gmt_features = [
    'orbit', 'bx', 'nStubs', 'stubHwPhi', 'stubHwPhiB', 'stubHwQual', 'stubHwEta', 'stubHwQEta', 'stubStation', 'stubWheel', 'stubSector', 'stubTag',
    'nBmtfMuons', 'bmtfHwPt', 'bmtfHwPtu', 'bmtfHwPhi', 'bmtfHwEta', 'bmtfPt', 'bmtfPtu', 'bmtfPhi', 'bmtfEta', 'bmtfHwSign', 'bmtfHwSignValid', 
    'bmtfHwDXY', 'bmtfHwQual', 'bmtfProcessor', 
    'nGmtMuons', 'gmtHwPt', 'gmtHwPtu', 'gmtHwPhi', 'gmtHwEta', 'gmtHwPhiAtVtx', 'gmtHwEtaAtVtx', 'gmtPt', 'gmtPtu', 'gmtPhi', 'gmtEta', 
    'gmtPhiAtVtx', 'gmtEtaAtVtx', 'gmtCharge', 'gmtDxy', 'gmtQual', 'gmtTfIndex']

stub_bmtf_gmt_list = []
for file in files:
    with uproot.open(file) as f:
        evs = f["bmtfNtuplizer/Events"]
        temp_awk = evs.arrays(stub_bmtf_gmt_features, library="ak")
        stub_bmtf_gmt_list.append(temp_awk)
        
bmtf_gmt = ak.concatenate(stub_bmtf_gmt_list, axis=0)



In [3]:
bx = bmtf_gmt.bx
nStubs = bmtf_gmt.nStubs
stubsStation = bmtf_gmt.stubStation
stubsSector = bmtf_gmt.stubSector
stubsWheel = bmtf_gmt.stubWheel
stubsPhiB = bmtf_gmt.stubHwPhiB
stubsHwQual = bmtf_gmt.stubHwQual

In [4]:
#123, 258, 287, 
'''for i in range(50):
   print(i, nStubs[i], stubsStation[i], stubsSector[i], stubsWheel[i], stubsHwQual[i])'''

'''i, j = 123,124
print(i, nStubs[i], stubsStation[i], stubsSector[i], stubsWheel[i])
print(j, nStubs[j], stubsStation[j], stubsSector[j], stubsWheel[j])'''

'i, j = 123,124\nprint(i, nStubs[i], stubsStation[i], stubsSector[i], stubsWheel[i])\nprint(j, nStubs[j], stubsStation[j], stubsSector[j], stubsWheel[j])'

In [5]:
Features = [nStubs, stubsStation, stubsSector, stubsWheel, stubsPhiB, stubsHwQual]

In [6]:
#Controlla che la stub successiva si trovi nella stazione dopo
def CheckStation(Station, StationP1):

   Diff = Station - StationP1
   Index = ak.where(Diff == -1)[0]

   return Index

#Controlla che la Wheel successiva sia +- 1 della wheel corrente, tranne per le wheel estremali
def CheckWheel(Wheel, WheelP1):
   
   IndexList = []

   if (Wheel == -2):
      Index = list(ak.where((WheelP1 == -1) | (WheelP1 == -2))[0])
      IndexList.extend(Index)

   elif (Wheel == 2):
      Index = list(ak.where((WheelP1 == 1) | (WheelP1 == 2))[0])
      IndexList.extend(Index)

   else:
      Diff = Wheel - WheelP1
      Index = list(ak.where((Diff == -1) | (Diff == 0) | (Diff == 1))[0])
      IndexList.extend(Index)

   return ak.Array(IndexList)

#Controlla che il settore successivo sia adiacente al settore corrente. Applica operazione ciclica per settori 11 e 0
def CheckSector(Sector, SectorP1):
   
   IndexList = []

   if (Sector == 11):
      Index = list(ak.where((SectorP1 == 10) | (SectorP1 == 11) | (SectorP1 == 0))[0])
      IndexList.extend(Index)

   elif (Sector == 0):
      Index = list(ak.where((SectorP1 == 11) | (SectorP1 == 0)| (SectorP1 == 1))[0])
      IndexList.extend(Index)

   else:
      Diff = Sector - SectorP1
      Index = list(ak.where((Diff == -1) | (Diff == 0) | (Diff == 1))[0])
      IndexList.extend(Index)

   return ak.Array(IndexList)

In [102]:
def SearchHSCP_StationLevel(Features, Number):
   
   Property = []

   #Feature: 0 == Stubs, 1 == Station, 2 == Sector, 3 == Wheel, 4 == PhiB, 5 == Quality

   for i in tqdm.tqdm(range(Number - 1)):

      Station, Sector, Wheel, PhiB, Quality  = [], [], [], [], []
      StationP1, SectorP1, WheelP1, PhiBP1, QualityP1 = [], [], [], [], []
      BoolChkStation = []
      
      #number of elements
      NoE = []
      

      Dic = {}
      DicP1 = {}

      #Inizializzo dizionari
      for feature in ['Station', 'Sector', 'Wheel', 'PhiB', 'Quality']:
         Dic[feature] = []
        
      for features in ['Station', 'Sector', 'Wheel', 'PhiB', 'Quality', 'NoE']:
         DicP1[features] = []

      
      if ((Features[0][i] == 1 and Features[0][i+1] == 1) and Features[1][i+1] != Features[1][i]+1):
         continue
      
      #Caso in cui ci sia solo 1 elemento nell'iesimo BX e 1 elemento nel BX successivo
      elif ((Features[0][i] == 1 and Features[0][i+1] == 1) and Features[1][i+1] == Features[1][i]+1):
         nStubs = [Features[0][i], Features[0][i+1]]    

         Dic['Station'].extend(Features[1][i])
         Dic['Sector'].extend(Features[2][i])
         Dic['Wheel'].extend(Features[3][i])
         Dic['PhiB'].extend(Features[4][i])
         Dic['Quality'].extend(Features[5][i])

         DicP1['Station'].extend(Features[1][i+1])
         DicP1['Sector'].extend(Features[2][i+1])
         DicP1['Wheel'].extend(Features[3][i+1])
         DicP1['PhiB'].extend(Features[4][i+1])
         DicP1['Quality'].extend(Features[5][i+1])
         DicP1['NoE'].append(1)

         Dictionary = {
            'i': i,
            'nStubs': nStubs,
            'property': Dic,
            'propertyP1': DicP1
         }

         Property.append(Dictionary)

      
      #Controllo duplicati viene fatto più avanti per ridurre i missmatch
      else:

         if (Features[0][i] > Features[0][i+1]):
            for j in range(Features[0][i]):
               ChkStation = CheckStation(Features[1][i][j], Features[1][i+1])
               
               if (len(ChkStation) != 0):
                  nStubs = [Features[0][i], Features[0][i+1]]
                  Station.append(Features[1][i][j])
                  Sector.append(Features[2][i][j])
                  Wheel.append(Features[3][i][j])
                  PhiB.append(Features[4][i][j])
                  Quality.append(Features[5][i][j])

                  StationP1.extend([Features[1][i+1][k] for k in ChkStation])
                  SectorP1.extend([Features[2][i+1][k] for k in ChkStation])
                  WheelP1.extend([Features[3][i+1][k] for k in ChkStation])
                  PhiBP1.extend([Features[4][i+1][k] for k in ChkStation])
                  QualityP1.extend([Features[5][i+1][k] for k in ChkStation])
                  NoE.append(len(ChkStation))

                  Dic['Station'].extend(Station)
                  Dic['Sector'].extend(Sector)
                  Dic['Wheel'].extend(Wheel)
                  Dic['PhiB'].extend(PhiB)
                  Dic['Quality'].extend(Quality)

                  DicP1['Station'].extend(StationP1)
                  DicP1['Sector'].extend(SectorP1)
                  DicP1['Wheel'].extend(WheelP1)   
                  DicP1['PhiB'].extend(PhiBP1)
                  DicP1['Quality'].extend(QualityP1)
                  DicP1['NoE'].extend([55555])



                  Dictionary = {
                     'i': i,
                     'nStubs': nStubs,
                     'property': Dic,
                     'propertyP1': DicP1
                  }


                  Property.append(Dictionary)


         else:
            #Tutti gli altri casi
            for j in range(Features[0][i]):
               ChkStation = CheckStation(Features[1][i][j], Features[1][i+1])
               nStubs = [Features[0][i], Features[0][i+1]]


               Station.append(Features[1][i][j])
               Sector.append(Features[2][i][j])
               Wheel.append(Features[3][i][j])
               PhiB.append(Features[4][i][j])
               Quality.append(Features[5][i][j])

               
               StationP1.extend([Features[1][i+1][k] for k in ChkStation])
               SectorP1.extend([Features[2][i+1][k] for k in ChkStation])
               WheelP1.extend([Features[3][i+1][k] for k in ChkStation])
               PhiBP1.extend([Features[4][i+1][k] for k in ChkStation])
               QualityP1.extend([Features[5][i+1][k] for k in ChkStation])
               NoE.append(len(ChkStation))


               if (len(ChkStation) == 0):
                  BoolChkStation.append(False)
               else:
                  BoolChkStation.append(True)

            if (np.any(np.array(BoolChkStation))):
               Dic['Station'].extend(Station)
               Dic['Sector'].extend(Sector)
               Dic['Wheel'].extend(Wheel)
               Dic['PhiB'].extend(PhiB)
               Dic['Quality'].extend(Quality)

               DicP1['Station'].extend(StationP1)
               DicP1['Sector'].extend(SectorP1)
               DicP1['Wheel'].extend(WheelP1)   
               DicP1['PhiB'].extend(PhiBP1)
               DicP1['Quality'].extend(QualityP1)
               DicP1['NoE'].extend(NoE)



               Dictionary = {
                  'i': i,
                  'nStubs': nStubs,
                  'property': Dic,
                  'propertyP1': DicP1
               }


               Property.append(Dictionary)


   return ak.Array(Property)
   

In [103]:
StationFiltered = SearchHSCP_StationLevel(Features, 1_000)#len(Features[0]))

100%|██████████| 999/999 [00:00<00:00, 1055.12it/s]


In [105]:
StationFiltered[3]
#StationFiltered[4]['propertyP1'].NoE

<Record {i: 29, nStubs: [2, ...], ...} type='{i: int64, nStubs: var * int64...'>

In [9]:
# with open('StationFiltered.pkl', 'wb') as f:
#    pickle.dump(StationFiltered, f)

# with open('StationFiltered.pkl', 'rb') as f:
#     StationFiltered = pickle.load(f)

In [96]:
i = 29
j = 0
Features[1][i][j]

1

In [82]:
#68, 
Number = np.where(StationFiltered['i'] == 25)[0]
#print(Number)
StationFiltered[Number[0]]

#print(StationFiltered[Number[0]]['property']['Station'][0])
#print(StationFiltered[Number[0]]['propertyP1']['Station'])

Station, Sector, Wheel, PhiB, Quality  = [], [], [], [], []
StationP1, SectorP1, WheelP1, PhiBP1, QualityP1 = [], [], [], [], []
BoolChkStation = []

Dic, DicP1 = {}, {}


for feature in ['Station', 'Sector', 'Wheel', 'PhiB', 'Quality']:
   Dic[feature] = []
   DicP1[feature] = []


i = 29
for j in range(2):
   ChkStation = CheckStation(StationFiltered[Number[0]]['property']['Station'][j], StationFiltered[Number[0]]['propertyP1']['Station'])
   if (len(ChkStation) != 0):
      Station.append(Features[1][i][j])
      Sector.append(Features[2][i][j])
      Wheel.append(Features[3][i][j])
      PhiB.append(Features[4][i][j])
      Quality.append(Features[5][i][j])

      StationP1.extend([Features[1][i+1][k] for k in ChkStation])
      SectorP1.extend([Features[2][i+1][k] for k in ChkStation])
      WheelP1.extend([Features[3][i+1][k] for k in ChkStation])
      PhiBP1.extend([Features[4][i+1][k] for k in ChkStation])
      QualityP1.extend([Features[5][i+1][k] for k in ChkStation])

      Dic['Station'].extend(Station)
      Dic['Sector'].extend(Sector)
      Dic['Wheel'].extend(Wheel)
      Dic['PhiB'].extend(PhiB)
      Dic['Quality'].extend(Quality)

      DicP1['Station'].extend(StationP1)
      DicP1['Sector'].extend(SectorP1)
      DicP1['Wheel'].extend(WheelP1)   
      DicP1['PhiB'].extend(PhiBP1)
      DicP1['Quality'].extend(QualityP1)

      Dictionary = {
      'i': i,
      'property': Dic,
      'propertyP1': DicP1
   }


In [83]:
Dictionary

{'i': 29,
 'property': {'Station': [1],
  'Sector': [3],
  'Wheel': [0],
  'PhiB': [206],
  'Quality': [4]},
 'propertyP1': {'Station': [2],
  'Sector': [2],
  'Wheel': [2],
  'PhiB': [-84],
  'Quality': [3]}}

In [11]:
# for i in range(1000):
#    if ak.all(StationFiltered[i]['nStubs'] == 2):
#       print(StationFiltered[i]['i'])

In [12]:
i = 2

Sector, SectorP1  = StationFiltered[i]['property'].Sector, StationFiltered[i]['propertyP1'].Sector
Wheel, WheelP1 = StationFiltered[i]['property'].Wheel, StationFiltered[i]['propertyP1'].Wheel

j = 0
IndexSector = CheckSector(Sector[j], SectorP1)

IndexWheel = CheckWheel(Wheel[j], WheelP1)

IndexSector

<Array [0] type='1 * int64'>

In [ ]:
#Creare una maschera che nasconda gli elementi che non sono conformi alla ricerca di HSCP o Muoni cosmici
def CosmicMuons(StationFiltered, Number):

   Mask = []

   for i in tqdm.tqdm(range(Number)):

      Station, StationP1  = StationFiltered[i]['property'].Station, StationFiltered[i]['propertyP1'].Station
      Sector, SectorP1  = StationFiltered[i]['property'].Sector, StationFiltered[i]['propertyP1'].Sector
      Wheel, WheelP1 = StationFiltered[i]['property'].Wheel, StationFiltered[i]['propertyP1'].Wheel
      
      if (StationFiltered[i]['nStubs'][0] == 1 and StationFiltered[i]['nStubs'][1] >= 1):

         IndexSector = CheckSector(Sector, SectorP1)
         IndexWheel = CheckWheel(Wheel, WheelP1)
        
         if (len(IndexSector) != 0 and len(IndexWheel) != 0):
            Mask.append(i)

      elif (StationFiltered[i]['nStubs'][0] > StationFiltered[i]['nStubs'][1]):
         for j in range(StationFiltered[i]['nStubs'][0]):
            ChkStation = CheckStation(Station[j], StationP1)

            if (len(ChkStation != 0)):
               IndexSector = CheckSector(Sector[j], SectorP1)
               IndexWheel = CheckWheel(Wheel[j], WheelP1)

               if (len(IndexSector) != 0 and len(IndexWheel) != 0):
                  Mask.append(i)


      else:
         for j in range(StationFiltered[i]['nStubs'][0]):
            IndexSector = CheckSector(Sector[j], SectorP1)
            IndexWheel = CheckWheel(Wheel[j], WheelP1)

            if (len(IndexSector) != 0 and len(IndexWheel) != 0):
               Mask.append(i)

         
   return StationFiltered[Mask]

In [14]:
test = CosmicMuons(StationFiltered, 2000)

100%|██████████| 2000/2000 [00:09<00:00, 217.75it/s]


In [17]:
test[0]

<Record {i: 25, nStubs: [2, ...], ...} type='{i: int64, nStubs: var * int64...'>